# TSA Chapter 2: AR(1) Process Properties

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch2/TSA_ch2_ar1/TSA_ch2_ar1.ipynb)

This notebook demonstrates:
- AR(1) model properties: mean mu = c/(1-phi), variance gamma(0) = sigma^2/(1-phi^2), ACF geometric decay rho(h) = phi^h
- Simulation with different phi values

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Set random seed

def simulate_ar1(n, phi, c=0, sigma=1):
    """Simulate AR(1) process"""
    x = np.zeros(n)
    eps = np.random.normal(0, sigma, n)
    for t in range(1, n):
        x[t] = c + phi * x[t-1] + eps[t]
    return x

n = 500

print("=" * 60)
print("AR(1) PROCESS: X_t = c + φX_{t-1} + ε_t")
print("=" * 60)

print("""
Stationarity Condition: |φ| < 1

If stationary:
  Mean:     μ = c / (1 - φ)
  Variance: γ(0) = σ² / (1 - φ²)
  ACF:      ρ(h) = φ^h  (exponential decay)
""")

# Different phi values
phi_values = [0.9, 0.5, -0.7, 0.99]

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, phi in enumerate(phi_values):
    # Simulate
    x = simulate_ar1(n, phi, c=0, sigma=1)

    # Time series plot
    axes[0, i].plot(x, 'b-', linewidth=0.5, alpha=0.8)
    axes[0, i].axhline(y=0, color='red', linestyle='--', linewidth=1)
    axes[0, i].set_title(f'AR(1): φ = {phi}', fontsize=11)
    axes[0, i].set_xlabel('Time')
    axes[0, i].set_ylabel('X_t')
    axes[0, i].grid(True, alpha=0.3)

    # Theoretical variance
    if abs(phi) < 1:
        var_theory = 1 / (1 - phi**2)
        axes[0, i].axhline(y=2*np.sqrt(var_theory), color='gray', linestyle=':', alpha=0.5)
        axes[0, i].axhline(y=-2*np.sqrt(var_theory), color='gray', linestyle=':', alpha=0.5)

    # ACF plot
    acf_values = acf(x, nlags=20)
    theoretical_acf = phi ** np.arange(21)

    axes[1, i].bar(range(21), acf_values, color='blue', alpha=0.5, label='Sample ACF')
    axes[1, i].plot(range(21), theoretical_acf, 'r-o', markersize=4, linewidth=2, label='Theoretical')
    axes[1, i].axhline(y=1.96/np.sqrt(n), color='gray', linestyle='--')
    axes[1, i].axhline(y=-1.96/np.sqrt(n), color='gray', linestyle='--')
    axes[1, i].set_title(f'ACF: ρ(h) = φ^h = {phi}^h', fontsize=10)
    axes[1, i].set_xlabel('Lag')
    axes[1, i].set_ylabel('ACF')
    axes[1, i].legend(fontsize=8)
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
save_chart(fig, 'ch2_ar1_properties')
plt.show()

# Detailed calculations for one example
print("\n" + "=" * 60)
print("NUMERICAL EXAMPLE: AR(1) with φ = 0.7, c = 2, σ² = 9")
print("=" * 60)

phi = 0.7
c = 2
sigma_sq = 9

mu = c / (1 - phi)
gamma_0 = sigma_sq / (1 - phi**2)
gamma_1 = phi * gamma_0
rho_1 = phi

print(f"""
Given: c = {c}, φ = {phi}, σ² = {sigma_sq}

Mean:
  μ = c / (1-φ) = {c} / (1-{phi}) = {c} / {1-phi} = {mu:.2f}

Variance:
  γ(0) = σ² / (1-φ²) = {sigma_sq} / (1-{phi**2}) = {sigma_sq} / {1-phi**2:.2f} = {gamma_0:.2f}

Autocovariance at lag 1:
  γ(1) = φ × γ(0) = {phi} × {gamma_0:.2f} = {gamma_1:.2f}

Autocorrelation:
  ρ(1) = φ = {rho_1}
  ρ(2) = φ² = {phi**2}
  ρ(3) = φ³ = {phi**3:.3f}
""")

print("\n" + "=" * 60)
print("STATIONARITY REGIONS")
print("=" * 60)
print("""
φ = 1:    Unit root (random walk) - NON-STATIONARY
|φ| < 1:  Stationary (mean-reverting)
|φ| > 1:  Explosive - NON-STATIONARY

φ > 0:    Positive autocorrelation, smooth patterns
φ < 0:    Negative autocorrelation, oscillating patterns
φ ≈ 1:    Highly persistent, slow mean reversion
φ ≈ 0:    Weak dependence, close to white noise
""")